In [32]:
import torch
import torch.optim as optim

import argparse
import time
import random
import dsacstar
import os

from network import Network
import datasets
from utils import tr, reverse_tr
import pickle
from torch.utils.tensorboard import SummaryWriter

In [15]:
dataset = datasets.SevenScenesDataset(f'/mundus/mrahman527/projects/homography-loss-function/datasets/7-Scenes/fire', 0.025, 0.975)


Loading seq-01


100%|███████████████████████████████████████| 1000/1000 [01:18<00:00, 12.80it/s]


Loading seq-02


100%|███████████████████████████████████████| 1000/1000 [01:18<00:00, 12.71it/s]


Loading seq-03


100%|███████████████████████████████████████| 1000/1000 [01:14<00:00, 13.46it/s]


Loading seq-04


100%|███████████████████████████████████████| 1000/1000 [01:13<00:00, 13.63it/s]


Sorting depths, this may take a while...


In [24]:

train_dataset = datasets.RelocDataset(dataset.train_data)
test_dataset = datasets.RelocDataset(dataset.test_data)

trainset_loader = torch.utils.data.DataLoader(train_dataset, shuffle=False, num_workers=6, batch_size=1)
testset_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=6, batch_size=1)

# load network
network = Network(torch.zeros((3)), False)
with_init=False
network = network.cuda()
network.train()


optimizer = torch.optim.Adam(network.parameters(),lr=0.000001)
iteration = 2

if with_init:
    writer_folder = 'with_init'
else:
    writer_folder = 'without_init'
    
    
writer = SummaryWriter(os.path.join('logs',os.path.basename(os.path.normpath('7-Scenes')),'fire',writer_folder))


if with_init:
        checkpoint_folder = f'our_checkpoints/{opt.dataset_name}/{opt.scene_name}_with_init'
        os.mkdir(checkpoint_folder)
else:
    checkpoint_folder = f"our_checkpoints/{'7-Scenes'}/{'fire'}_without_init"
    if os.path.isdir(checkpoint_folder):
        checkpoint_folder = checkpoint_folder+'_1'

    os.makedirs(checkpoint_folder, exist_ok=True)


In [27]:
len(trainset_loader)

2000

In [35]:

def train(network = network,trainset_loader=trainset_loader,testset_laoder=testset_loader,optimizer=optimizer, iteration=iteration, with_init=with_init, writer=writer,checkpoint_folder=checkpoint_folder):
    network.train()
    for epoch in range(iteration):
        print(f'epoch:{epoch}\n')
        running_loss = 0
        it = 0
        for data in trainset_loader:
            it+=1
            with torch.no_grad():
                optimizer.zero_grad()

            focal_length = data['K'][0][0][0]
            file = data['image_file']
            image = data['image'].cuda()
#             start_time = time.time()
            wtc, crw = data['w_t_c'], data['c_R_w']
            
            # predict scene coordinates and neural guidance
            scene_coordinates = network(image)
            scene_coordinates_gradients = torch.zeros(scene_coordinates.size())
            gt_pose = reverse_tr(crw, wtc)[0]
            # print(f"shape pose={gt_pose.shape}")
            # print(f"xmin = {data['xmin']} shape {data['xmin'].shape}")
            # print(f"xmax = {data['xmax']} shape {data['xmax'].shape}")

            # pose from RGB
            loss = dsacstar.backward_rgb(
                scene_coordinates.cpu(),
                scene_coordinates_gradients,
                gt_pose, 
                64, 
                10,
                focal_length, 
                float(image.size(3) / 2), #principal point assumed in image center
                float(image.size(2) / 2),
                1.0,
                100.0,
                100,
                100,
                100,
                network.OUTPUT_SUBSAMPLE,
                random.randint(0,1000000), #used to initialize random number generator in cpp
                data['xmin'].item(),
                data['xmax'].item()
            )
    
            
            running_loss += loss
            torch.autograd.backward((scene_coordinates),(scene_coordinates_gradients.cuda()))
            optimizer.step()
            if it%10==0 and it!=0:
                writer.add_scalar('train loss',running_loss/it)
                
        
        writer.add_scalar('per_epoch_training_loss',running_loss/len(trainset_loader),epoch)
        

#         if epoch%opt.save_every==0:
#             checkpoint_path = os.path.join(checkpoint_folder,f'check_point_epoch_{epoch}.pt')
#             torch.save(
#                 {
#                     'epoch': epoch,
#                     'model_state_dict': network.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),

#                 }, checkpoint_path
#             )
       
        print(f"loss: {running_loss}")
    
    with open(os.path.join(checkpoint_folder,'loss_list.pickle'),'wb') as f:
        pickle.dump({'loss_list':loss_list,'per_epoch_loss_list':per_epoch_loss_list}, f)

    

train(network = network, optimizer=optimizer, iteration=iteration)



epoch:0

Sampling 64 hypotheses.
Done in 0.459959s.
Calculating scores.
Entropy: 5.99479
Done in 0.161203s.
Refining poses:
Done in 0.0479395s.
Calculating gradients wrt hypotheses.
Done in 0.0417846s.
Calculating gradients wrt scores.
Done in 1.06847s.
Sampling 64 hypotheses.
Done in 0.421603s.
Calculating scores.
Entropy: 5.98912
Done in 0.161269s.
Refining poses:
Done in 0.057662s.
Calculating gradients wrt hypotheses.
Done in 0.0292934s.
Calculating gradients wrt scores.
Done in 1.07494s.
Sampling 64 hypotheses.
Done in 0.347883s.
Calculating scores.
Entropy: 5.99226
Done in 0.14727s.
Refining poses:
Done in 0.0592006s.
Calculating gradients wrt hypotheses.
Done in 0.0287661s.
Calculating gradients wrt scores.
Done in 1.08382s.
Sampling 64 hypotheses.
Done in 0.430806s.
Calculating scores.
Entropy: 5.99317
Done in 0.145816s.
Refining poses:
Done in 0.0572331s.
Calculating gradients wrt hypotheses.
Done in 0.027827s.
Calculating gradients wrt scores.
Done in 1.08012s.
Sampling 64 hy

Sampling 64 hypotheses.
Done in 0.409239s.
Calculating scores.
Entropy: 5.99309
Done in 0.150925s.
Refining poses:
Done in 0.0617385s.
Calculating gradients wrt hypotheses.
Done in 0.0278727s.
Calculating gradients wrt scores.
Done in 1.0852s.
Sampling 64 hypotheses.
Done in 0.364519s.
Calculating scores.
Entropy: 5.99064
Done in 0.152223s.
Refining poses:
Done in 0.0581254s.
Calculating gradients wrt hypotheses.
Done in 0.0274019s.
Calculating gradients wrt scores.
Done in 1.0876s.
Sampling 64 hypotheses.
Done in 0.36033s.
Calculating scores.
Entropy: 5.99436
Done in 0.156855s.
Refining poses:
Done in 0.064701s.
Calculating gradients wrt hypotheses.
Done in 0.027825s.
Calculating gradients wrt scores.
Done in 1.07403s.
Sampling 64 hypotheses.
Done in 0.440272s.
Calculating scores.
Entropy: 5.99176
Done in 0.154584s.
Refining poses:
Done in 0.0576735s.
Calculating gradients wrt hypotheses.
Done in 0.0279355s.
Calculating gradients wrt scores.
Done in 1.07995s.
Sampling 64 hypotheses.
D

Sampling 64 hypotheses.
Done in 0.352835s.
Calculating scores.
Entropy: 5.98305
Done in 0.158678s.
Refining poses:
Done in 0.0595214s.
Calculating gradients wrt hypotheses.
Done in 0.0329719s.
Calculating gradients wrt scores.
Done in 1.10112s.
Sampling 64 hypotheses.
Done in 0.303308s.
Calculating scores.
Entropy: 5.9889
Done in 0.162714s.
Refining poses:
Done in 0.0569847s.
Calculating gradients wrt hypotheses.
Done in 0.0285621s.
Calculating gradients wrt scores.
Done in 1.11225s.
Sampling 64 hypotheses.
Done in 0.389749s.
Calculating scores.
Entropy: 5.9898
Done in 0.155721s.
Refining poses:
Done in 0.0565466s.
Calculating gradients wrt hypotheses.
Done in 0.0285806s.
Calculating gradients wrt scores.
Done in 1.13364s.
Sampling 64 hypotheses.
Done in 0.347782s.
Calculating scores.
Entropy: 5.98883
Done in 0.134944s.
Refining poses:
Done in 0.059407s.
Calculating gradients wrt hypotheses.
Done in 0.0284072s.
Calculating gradients wrt scores.
Done in 1.11126s.
Sampling 64 hypotheses.

Done in 0.405338s.
Calculating scores.
Entropy: 5.99079
Done in 0.144393s.
Refining poses:
Done in 0.0621339s.
Calculating gradients wrt hypotheses.
Done in 0.0297283s.
Calculating gradients wrt scores.
Done in 1.08892s.
Sampling 64 hypotheses.
Done in 0.310258s.
Calculating scores.
Entropy: 5.98555
Done in 0.14667s.
Refining poses:
Done in 0.0523662s.
Calculating gradients wrt hypotheses.
Done in 0.0282471s.
Calculating gradients wrt scores.
Done in 1.10005s.
Sampling 64 hypotheses.
Done in 0.33377s.
Calculating scores.
Entropy: 5.98975
Done in 0.151646s.
Refining poses:
Done in 0.0562141s.
Calculating gradients wrt hypotheses.
Done in 0.0280479s.
Calculating gradients wrt scores.
Done in 1.06648s.
Sampling 64 hypotheses.
Done in 0.323555s.
Calculating scores.
Entropy: 5.99059
Done in 0.157002s.
Refining poses:
Done in 0.0685963s.
Calculating gradients wrt hypotheses.
Done in 0.0280567s.
Calculating gradients wrt scores.
Done in 1.07615s.
Sampling 64 hypotheses.
Done in 0.334281s.
Cal

Sampling 64 hypotheses.
Done in 0.268279s.
Calculating scores.
Entropy: 5.97766
Done in 0.152058s.
Refining poses:
Done in 0.0754775s.
Calculating gradients wrt hypotheses.
Done in 0.0295813s.
Calculating gradients wrt scores.
Done in 1.07506s.
Sampling 64 hypotheses.
Done in 0.300017s.
Calculating scores.
Entropy: 5.98434
Done in 0.153902s.
Refining poses:
Done in 0.0763217s.
Calculating gradients wrt hypotheses.
Done in 0.0295213s.
Calculating gradients wrt scores.
Done in 1.08681s.
Sampling 64 hypotheses.
Done in 0.275567s.
Calculating scores.
Entropy: 5.98952
Done in 0.155691s.
Refining poses:
Done in 0.0649679s.
Calculating gradients wrt hypotheses.
Done in 0.0295659s.
Calculating gradients wrt scores.
Done in 1.0845s.
Sampling 64 hypotheses.
Done in 0.250631s.
Calculating scores.
Entropy: 5.97981
Done in 0.155312s.
Refining poses:
Done in 0.0700713s.
Calculating gradients wrt hypotheses.
Done in 0.0284043s.
Calculating gradients wrt scores.
Done in 1.08405s.
Sampling 64 hypothese

...

Sampling 64 hypotheses.
Done in 0.29734s.
Calculating scores.
Entropy: 5.96122
Done in 0.153238s.
Refining poses:
Done in 0.0646221s.
Calculating gradients wrt hypotheses.
Done in 0.0282982s.
Calculating gradients wrt scores.
Done in 1.07316s.
Sampling 64 hypotheses.
Done in 0.375246s.
Calculating scores.
Entropy: 5.97281
Done in 0.161517s.
Refining poses:
Done in 0.066785s.
Calculating gradients wrt hypotheses.
Done in 0.0309801s.
Calculating gradients wrt scores.
Done in 1.08506s.
Sampling 64 hypotheses.
Done in 0.372175s.
Calculating scores.
Entropy: 5.98357
Done in 0.159456s.
Refining poses:
Done in 0.0590933s.
Calculating gradients wrt hypotheses.
Done in 0.029375s.
Calculating gradients wrt scores.
Done in 1.06779s.
Sampling 64 hypotheses.
Done in 0.298509s.
Calculating scores.
Entropy: 5.9775
Done in 0.149675s.
Refining poses:
Done in 0.0589642s.
Calculating gradients wrt hypotheses.
Done in 0.0280856s.
Calculating gradients wrt scores.
Done in 1.07209s.
Sampling 64 hypotheses.


Sampling 64 hypotheses.
Done in 0.300575s.
Calculating scores.
Entropy: 5.9795
Done in 0.153875s.
Refining poses:
Done in 0.0604917s.
Calculating gradients wrt hypotheses.
Done in 0.028049s.
Calculating gradients wrt scores.
Done in 1.06891s.
Sampling 64 hypotheses.
Done in 0.273248s.
Calculating scores.
Entropy: 5.99066
Done in 0.149947s.
Refining poses:
Done in 0.0665714s.
Calculating gradients wrt hypotheses.
Done in 0.0280306s.
Calculating gradients wrt scores.
Done in 1.06948s.
Sampling 64 hypotheses.
Done in 0.360685s.
Calculating scores.
Entropy: 5.97954
Done in 0.148416s.
Refining poses:
Done in 0.0699193s.
Calculating gradients wrt hypotheses.
Done in 0.0274788s.
Calculating gradients wrt scores.
Done in 1.07381s.
Sampling 64 hypotheses.
Done in 0.366257s.
Calculating scores.
Entropy: 5.98569
Done in 0.152252s.
Refining poses:
Done in 0.0579083s.
Calculating gradients wrt hypotheses.
Done in 0.0276932s.
Calculating gradients wrt scores.
Done in 1.07425s.
Sampling 64 hypotheses

Sampling 64 hypotheses.
Done in 0.349117s.
Calculating scores.
Entropy: 5.9896
Done in 0.153225s.
Refining poses:
Done in 0.0610332s.
Calculating gradients wrt hypotheses.
Done in 0.0280259s.
Calculating gradients wrt scores.
Done in 1.05336s.
Sampling 64 hypotheses.
Done in 0.279757s.
Calculating scores.
Entropy: 5.98529
Done in 0.147299s.
Refining poses:
Done in 0.0682734s.
Calculating gradients wrt hypotheses.
Done in 0.0286899s.
Calculating gradients wrt scores.
Done in 1.06354s.
Sampling 64 hypotheses.
Done in 0.370766s.
Calculating scores.
Entropy: 5.99185
Done in 0.151208s.
Refining poses:
Done in 0.0704032s.
Calculating gradients wrt hypotheses.
Done in 0.0286802s.
Calculating gradients wrt scores.
Done in 1.06745s.
Sampling 64 hypotheses.
Done in 0.393651s.
Calculating scores.
Entropy: 5.98903
Done in 0.153432s.
Refining poses:
Done in 0.0625039s.
Calculating gradients wrt hypotheses.
Done in 0.0294493s.
Calculating gradients wrt scores.
Done in 1.06663s.
Sampling 64 hypothese

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/mundus/mrahman527/miniconda3/envs/dsacstar/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/mundus/mrahman527/miniconda3/envs/dsacstar/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/mundus/mrahman527/miniconda3/envs/dsacstar/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/mundus/mrahman527/miniconda3/envs/dsacstar/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [36]:
!nvidia-smi


Mon May  8 17:18:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:1B:00.0 Off |                  N/A |
|  0%   40C    P8               25W / 350W|   2781MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--